In [217]:
# Dependencies
import pandas as pd
#pd.options.mode.chained_assignment = None  # default='warn'

import json
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# pull in postgres password
import sys
sys.path.append("C:/Users/laura/bootcamp/")
from api_keys import db_password

# ZIP CODE DATA

In [218]:
# Store filepath in a variable
Zipcode = "Resources/Portland_Zip_and_Neighborhood.csv"

In [219]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zip_df = pd.read_csv(Zipcode, encoding="ISO-8859-1")
zip_df.head()

,Zip_Code,Neighborhood_Name
0,97212,Alameda
1,97217,Arbor Lodge
2,97222,Ardenwald
3,97230,Argay
4,97201,Arlington Heights


# AIRBNB DATA (no longer using but didn't want to delete code yet - Laura 4.7.22)

In [220]:
# Store filepath in a variable
ARBNB_old = "Resources/AB_US_2020.csv"

In [221]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df_old = pd.read_csv(ARBNB_old, encoding="ISO-8859-1")

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [222]:
# Show just the header
#ARBNB_df_old.head()

In [223]:
column_list = ARBNB_df_old.columns.values.tolist()
#column_list

In [224]:
#ARBNB_df_old.describe()

In [225]:
# Keep the rows that are Portland
ARBNB_Portland_filter_old=ARBNB_df_old.loc[(ARBNB_df_old["city"]=="Portland")]
#ARBNB_Portland_filter_old.head()

In [226]:
#determine if there are blank data points that need to be dropped
#ARBNB_Portland_filter_old.isna().sum()

In [227]:
# drop columns that do not aid in analysis
ARBNB_Portland_old=ARBNB_Portland_filter_old.drop(columns=['ï»¿id','name','host_id','host_name',"neighbourhood_group","last_review"])
#ARBNB_Portland_old

In [228]:
#determine if there are blank data points that need to be dropped
#ARBNB_Portland_old.isna().sum()

In [229]:
#decided to drop nan values from reviews_per_month, could also look at dropping column
#ARBNB_Portland_dropna_old=ARBNB_Portland_old.dropna()

In [230]:
#ARBNB_Portland_dropna_old.isna().sum()

# AIRBNB DATA FROM AIRBNB SITE

In [231]:
# Store filepath in a variable
ARBNB = "Resources/Portland_airbnb_InsideAirbnb.csv"

In [232]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df = pd.read_csv(ARBNB, encoding="ISO-8859-1", 
                       #converters={"host_response_rate" : string_to_float} - error won't let do on N/A
                      )

In [233]:
# Show just the header
ARBNB_df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,12899,https://www.airbnb.com/rooms/12899,20211220005848,2021-12-20,"Alberta Arts 2 bedroom suite, charming 1906 house","Please know that we are fully vaccinated, will...",We're within walking distance of a grocery and...,https://a0.muscache.com/pictures/225005/cf0083...,49682,https://www.airbnb.com/users/show/49682,Ali And David,2009-10-29,"Portland, Oregon, United States","We enjoy cooking, singing, hiking, bicycles, t...",within an hour,100%,100%,t,https://a0.muscache.com/im/users/49682/profile...,https://a0.muscache.com/im/users/49682/profile...,Concordia,1,1,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,f,"Portland, Oregon, United States",Concordia,NaN,45.56488,-122.63418,Entire rental unit,Entire home/apt,3,NaN,1 bath,2.0,2.0,"[""Portable fans"", ""Security cameras on propert...",$89.00,3,730,3.0,3.0,1125.0,1125.0,3.0,1125.0,NaN,t,22,52,82,353,2021-12-20,591,40,1,2010-01-24,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,14-218887-000-00-HO,t,1,1,0,0,4.08
1,37676,https://www.airbnb.com/rooms/37676,20211220005848,2021-12-20,Mt. Hood View in the Pearl District,"This 1,000 SF loft is located in the heart of ...",The Pearl district enjoys a walkability score ...,https://a0.muscache.com/pictures/212298/16fb6b...,162158,https://www.airbnb.com/users/show/162158,Paul,2010-07-09,"Portland, Oregon, United States",I look forward to helping people discover the ...,within a day,100%,55%,t,https://a0.muscache.com/im/users/162158/profil...,https://a0.muscache.com/im/users/162158/profil...,Pearl,2,2,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"Portland, Oregon, United States",Pearl,NaN,45.52564,-122.68273,Entire loft,Entire home/apt,3,NaN,1 bath,1.0,1.0,"[""Single level home"", ""Coffee maker"", ""Elevato...",$140.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,30,60,90,365,2021-12-20,127,4,1,2010-07-22,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,NaN,f,1,1,0,0,0.91
2,41601,https://www.airbnb.com/rooms/41601,20211220005848,2021-12-20,Grandpa's Bunkhouse-Backyard Studio,Our quiet backyard cabin: Vaulted ceiling. Fre...,Ours is a neighborhood just off Noodle Flat in...,https://a0.muscache.com/pictures/294ac44c-168d...,179045,https://www.airbnb.com/users/show/179045,Jean,2010-07-24,"Portland, Oregon, United States",Jean is all about thrifty sustainable living a...,within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/480c9...,https://a0.muscache.com/im/pictures/user/480c9...,Roseway,2,2,"['email', 'phone', 'reviews', 'kba']",t,t,"Portland, Oregon, United States",Roseway,NaN,45.54804,-122.58541,Entire residential home,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Coffee maker"", ""Cooking basics"", ""Free stree...",$129.

In [234]:
ARBNB_df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
name                                             object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 74, dtype: object

In [235]:
column_list = ARBNB_df.columns.values.tolist()
column_list

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availa

In [236]:
# keep columns to be used in analysis
ARBNB_df2=ARBNB_df[[
     #'id',
     'listing_url',
     #'scrape_id',
     #'last_scraped',
     #'name',
     #'description',
     #'neighborhood_overview',
     #'picture_url',
     #'host_id',
     #'host_url',
     #'host_name',
     #'host_since',
     #'host_location',
     #'host_about',
     'host_response_time',
     'host_response_rate',
     'host_acceptance_rate',
     'host_is_superhost',
     #'host_thumbnail_url',
     #'host_picture_url',
     #'host_neighbourhood',
     #'host_listings_count',
     #'host_total_listings_count',
     #'host_verifications',
     #'host_has_profile_pic',
     'host_identity_verified',
     #'neighbourhood',
     'neighbourhood_cleansed',
     #'neighbourhood_group_cleansed',
     'latitude',
     'longitude',
     #'property_type',
     'room_type',
     'accommodates',
     #'bathrooms' - unfortunately this is blank so bathrooms_text will need to be cleaned further,
     'bathrooms_text',
     'bedrooms',
     'beds',
     #'amenities',
     'price',
     'minimum_nights',
     'maximum_nights',
     #'minimum_minimum_nights',
     #'maximum_minimum_nights',
     #'minimum_maximum_nights',
     #'maximum_maximum_nights',
     #'minimum_nights_avg_ntm',
     #'maximum_nights_avg_ntm',
     #'calendar_updated',
     #'has_availability',  (could decide to keep availability in)
     #'availability_30',
     #'availability_60',
     #'availability_90',
     #'availability_365',
     #'calendar_last_scraped',
     #'number_of_reviews',
     #'number_of_reviews_ltm',
     'number_of_reviews_l30d',
     #'first_review',
     'last_review',
     'review_scores_rating',
     'review_scores_accuracy',
     'review_scores_cleanliness',
     'review_scores_checkin',
     'review_scores_communication',
     'review_scores_location',
     'review_scores_value',
     #'license',
     #'instant_bookable',
     #'calculated_host_listings_count',
     #'calculated_host_listings_count_entire_homes',
     #'calculated_host_listings_count_private_rooms',
     #'calculated_host_listings_count_shared_rooms',
     'reviews_per_month'
]]
pd.set_option('display.max_columns', None)
ARBNB_df2.head()

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,t,f,Concordia,45.56488,-122.63418,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,t,t,Pearl,45.52564,-122.68273,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,t,t,Roseway,45.54804,-122.58541,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,t,t,Roseway,45.54793,-122.58358,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,t,t,Goose Hollow,45.52258,-122.69955,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24


In [237]:
#set new index - not needed with Katie's code on t and f
#ARBNB_df2_index=ARBNB_df2.set_index("id")

In [238]:
#filter to only look at entire home/apt
ARBNB_Portland_filter=ARBNB_df2.loc[(ARBNB_df2["room_type"]=="Entire home/apt")]
ARBNB_Portland_filter

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,t,f,Concordia,45.564880,-122.634180,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,t,t,Pearl,45.525640,-122.682730,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,t,t,Roseway,45.548040,-122.585410,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,t,t,Roseway,45.547930,-122.583580,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,t,t,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,f,t,Northwest District,45.531189,-122.690158,Entire home/apt,2,1 bath,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,f,t,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1 bath,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,t,f,Northwest District,45.534796,-122.697880,Entire home/apt,4,2 baths,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,t,t,King,45.561982,-122.661731,Entire home/apt,4,2.5 baths,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
# convert t/f and true/false to boolean int
def change_tf(value):
    if value == "t":
        return 1
    else:
        return 0
ARBNB_Portland_filter['host_is_superhost'] = ARBNB_Portland_filter['host_is_superhost'].apply(change_tf)
ARBNB_Portland_filter['host_identity_verified'] = ARBNB_Portland_filter['host_identity_verified'].apply(change_tf)
ARBNB_Portland_filter.head()

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.56488,-122.63418,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.52564,-122.68273,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.54804,-122.58541,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.54793,-122.58358,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24


In [240]:
#drop half-bath description since it is one data point and doesn't give a number
ARBNB_Portland_filter2=ARBNB_Portland_filter[ARBNB_Portland_filter.bathrooms_text != "Half-bath"]
ARBNB_Portland_filter2

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1 bath,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1 bath,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2 baths,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5 baths,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
# Display an overview of the bathrooms_text column
ARBNB_Portland_filter2['bathrooms_text'].value_counts()

1 bath       2373
2 baths       332
1.5 baths     141
2.5 baths      91
3 baths        55
3.5 baths      17
4 baths         7
5 baths         2
6.5 baths       1
Name: bathrooms_text, dtype: int64

In [242]:
# Clean up bathrooms_text category. 
ARBNB_Portland_filter2['bathrooms_text'] = ARBNB_Portland_filter2['bathrooms_text'].replace(
    {'1 bath': 1, '1.5 baths': 1.5, '2 baths': 2, "2.5 baths": 2.5, "3 baths": 3,
     "3.5 baths":3.5, "4 baths":4, "5 baths": 5, "6.5 baths": 6.5 })
ARBNB_Portland_filter2

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1.0,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1.0,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1.0,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2.0,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
# Clean up bathrooms_text category using iloc
ARBNB_Portland_filter2['bathrooms_text'] = ARBNB_Portland_filter2['bathrooms_text'].replace(
    {'1 bath': 1, '1.5 baths': 1.5, '2 baths': 2, "2.5 baths": 2.5, "3 baths": 3,
     "3.5 baths":3.5, "4 baths":4, "5 baths": 5, "6.5 baths": 6.5 })
ARBNB_Portland_filter2

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1.0,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1.0,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1.0,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2.0,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
ARBNB_Portland_filter2['bathrooms_text'].value_counts()

1.0    2373
2.0     332
1.5     141
2.5      91
3.0      55
3.5      17
4.0       7
5.0       2
6.5       1
Name: bathrooms_text, dtype: int64

In [245]:
# changes bathrooms_text to bathrooms
ARBNB_Portland_filter2.rename(columns = {'bathrooms_text':'bathrooms'}, inplace = True)
ARBNB_Portland_filter2

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1.0,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1.0,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1.0,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2.0,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
ARBNB_Portland_filter2.isna().sum()

listing_url                      0
host_response_time             316
host_response_rate             316
host_acceptance_rate           192
host_is_superhost                0
host_identity_verified           0
neighbourhood_cleansed           0
latitude                         0
longitude                        0
room_type                        0
accommodates                     0
bathrooms                        0
bedrooms                       409
beds                            35
price                            0
minimum_nights                   0
maximum_nights                   0
number_of_reviews_l30d           0
last_review                    258
review_scores_rating           258
review_scores_accuracy         261
review_scores_cleanliness      261
review_scores_checkin          261
review_scores_communication    261
review_scores_location         261
review_scores_value            261
reviews_per_month              258
dtype: int64

In [247]:
ARBNB_Portland_dropna=ARBNB_Portland_filter2.dropna()
ARBNB_Portland_dropna.describe()

,host_is_superhost,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
count,2101.00000,2101.00000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000
mean,0.68634,0.84436,45.529788,-122.651957,4.013803,1.257258,1.646835,2.112327,7.279867,531.967634,1.907663,4.852432,4.886069,4.859710,4.921266,4.915988,4.881447,4.795940,2.738696
std,0.46409,0.36260,0.032884,0.043276,2.033599,0.539489,0.962623,1.304478,11.577759,517.294375,2.232881,0.285615,0.276352,0.282657,0.274738,0.271439,0.234751,0.306419,2.267940
min,0.00000,0.00000,45.436120,-122.832860,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.010000
25%,0.00000,1.00000,45.508980,-122.680300,2.000000,1.000000,1.000000,1.000000,1.000000,30.000000,0.000000,4.830000,4.880000,4.840000,4.940000,4.920000,4.860000,4.750000,0.920000
50%,1.00000,1.00000,45.528520,-122.651370,4.000000,1.000000,1.000000,2.000000,2.000000,360.000000,1.000000,4.920000,4.950000,4.930000,4.980000,4.980000,4.930000,4.860000,2.240000
75%,1.00000,1.00000,45.556690,-122.626320,5.000000,1.000000,2.000000,2.000000,4.000000,1125.000000,3.000000,4.980000,5.000000,4.990000,5.000000,5.000000,4.990000,4.930000,3.930000
max,1.00000,1.00000,45.612050,-122.496730,16.000000,5.000000,9.000000,16.000000,120.000000,1125.000000,14.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,14.690000


In [248]:
# convert dates from object to date
ARBNB_Portland_dropna["last_review"]=ARBNB_Portland_dropna["last_review"].apply(pd.to_datetime)
ARBNB_Portland_dropna.head()

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.56488,-122.63418,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.52564,-122.68273,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.54804,-122.58541,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100%,0%,0,1,Sullivan's Gulch,45.53103,-122.64448,Entire home/apt,12,2.0,4.0,5.0,$425.00,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23


In [249]:
ARBNB_Portland_dropna.isna().sum()

listing_url                    0
host_response_time             0
host_response_rate             0
host_acceptance_rate           0
host_is_superhost              0
host_identity_verified         0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
room_type                      0
accommodates                   0
bathrooms                      0
bedrooms                       0
beds                           0
price                          0
minimum_nights                 0
maximum_nights                 0
number_of_reviews_l30d         0
last_review                    0
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
reviews_per_month              0
dtype: int64

In [250]:
ARBNB_Portland_dropna.dtypes

listing_url                            object
host_response_time                     object
host_response_rate                     object
host_acceptance_rate                   object
host_is_superhost                       int64
host_identity_verified                  int64
neighbourhood_cleansed                 object
latitude                              float64
longitude                             float64
room_type                              object
accommodates                            int64
bathrooms                             float64
bedrooms                              float64
beds                                  float64
price                                  object
minimum_nights                          int64
maximum_nights                          int64
number_of_reviews_l30d                  int64
last_review                    datetime64[ns]
review_scores_rating                  float64
review_scores_accuracy                float64
review_scores_cleanliness         

In [251]:
## no longer needed used Katie's code
# #change host_is_superhost to boolean and host_identity_verified to boolean
# d={"t":True,"f":False}
# ARBNB_Portland_b1=ARBNB_Portland_dropna['host_is_superhost'].map(d)
# ARBNB_Portland_b1

In [252]:
## no longer needed used Katie's code
# #change host_identity_verified to boolean
# ARBNB_Portland_b2=ARBNB_Portland_dropna['host_identity_verified'].map(d)

In [253]:
## no longer needed used Katie's code
# join_df=pd.concat([ARBNB_Portland_dropna, ARBNB_Portland_b1],axis=1)
# join_df

In [254]:
## no longer needed used Katie's code
# join_df.columns = [*join_df.columns[:-1], 'is_superhost']
# join_df

In [255]:
## no longer needed used Katie's code
# join_df2=pd.concat([join_df, ARBNB_Portland_b2],axis=1)
# join_df2

In [256]:
## no longer needed used Katie's code
# join_df2.columns = [*join_df2.columns[:-1], 'is_verified']
# join_df2

In [257]:
## no longer needed used Katie's code
# join_df2.drop(columns=["host_is_superhost", "host_identity_verified"])

In [258]:
#strip price of $ commma and .00 from price
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.strip("$")
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.replace(",","")
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.strip(".00")
ARBNB_Portland_dropna

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,89,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,14,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,129,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,104,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100%,0%,0,1,Sullivan's Gulch,45.531030,-122.644480,Entire home/apt,12,2.0,4.0,5.0,425,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100%,100%,0,1,Pearl,45.529118,-122.680400,Entire home/apt,3,1.0,1.0,2.0,233,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100%,86%,0,1,Maywood Park,45.553293,-122.561127,Entire home/apt,4,1.0,1.0,1.0,64,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100%,100%,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,2,1.0,1.0,1.0,137,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100%,100%,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,7,2.5,3.0,3.0,203,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [259]:
#strip host_response_rate of %
ARBNB_Portland_dropna["host_response_rate"]=ARBNB_Portland_dropna["host_response_rate"].str.replace("%","")
ARBNB_Portland_dropna

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,89,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,14,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,129,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,104,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100,0%,0,1,Sullivan's Gulch,45.531030,-122.644480,Entire home/apt,12,2.0,4.0,5.0,425,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100,100%,0,1,Pearl,45.529118,-122.680400,Entire home/apt,3,1.0,1.0,2.0,233,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100,86%,0,1,Maywood Park,45.553293,-122.561127,Entire home/apt,4,1.0,1.0,1.0,64,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100,100%,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,2,1.0,1.0,1.0,137,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100,100%,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,7,2.5,3.0,3.0,203,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [260]:
#strip host_acceptance_rate of %
ARBNB_Portland_dropna["host_acceptance_rate"]=ARBNB_Portland_dropna["host_acceptance_rate"].str.replace("%","")
ARBNB_Portland_dropna

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100,100,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,89,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100,55,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,14,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100,100,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,129,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100,73,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,104,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100,0,0,1,Sullivan's Gulch,45.531030,-122.644480,Entire home/apt,12,2.0,4.0,5.0,425,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100,100,0,1,Pearl,45.529118,-122.680400,Entire home/apt,3,1.0,1.0,2.0,233,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100,86,0,1,Maywood Park,45.553293,-122.561127,Entire home/apt,4,1.0,1.0,1.0,64,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100,100,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,2,1.0,1.0,1.0,137,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100,100,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,7,2.5,3.0,3.0,203,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [261]:
#convert to float
ARBNB_Portland_dropna["host_response_rate"]=ARBNB_Portland_dropna["host_response_rate"].astype(float)
ARBNB_Portland_dropna["host_acceptance_rate"]=ARBNB_Portland_dropna["host_acceptance_rate"].astype(float)
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].astype(float)

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [262]:
ARBNB_Portland_dropna.drop(columns=["room_type"])

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100.0,100.0,1,0,Concordia,45.564880,-122.634180,3,1.0,2.0,2.0,89.0,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100.0,55.0,1,1,Pearl,45.525640,-122.682730,3,1.0,1.0,1.0,14.0,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100.0,100.0,1,1,Roseway,45.548040,-122.585410,2,1.0,1.0,1.0,129.0,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100.0,73.0,1,1,Goose Hollow,45.522580,-122.699550,2,1.0,1.0,1.0,104.0,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100.0,0.0,0,1,Sullivan's Gulch,45.531030,-122.644480,12,2.0,4.0,5.0,425.0,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100.0,100.0,0,1,Pearl,45.529118,-122.680400,3,1.0,1.0,2.0,233.0,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100.0,86.0,0,1,Maywood Park,45.553293,-122.561127,4,1.0,1.0,1.0,64.0,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100.0,100.0,0,1,Southwest Hills,45.494583,-122.707946,2,1.0,1.0,1.0,137.0,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100.0,100.0,0,1,Humboldt,45.555253,-122.666735,7,2.5,3.0,3.0,203.0,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [263]:
ARBNB_Portland_dropna.dtypes

listing_url                            object
host_response_time                     object
host_response_rate                    float64
host_acceptance_rate                  float64
host_is_superhost                       int64
host_identity_verified                  int64
neighbourhood_cleansed                 object
latitude                              float64
longitude                             float64
room_type                              object
accommodates                            int64
bathrooms                             float64
bedrooms                              float64
beds                                  float64
price                                 float64
minimum_nights                          int64
maximum_nights                          int64
number_of_reviews_l30d                  int64
last_review                    datetime64[ns]
review_scores_rating                  float64
review_scores_accuracy                float64
review_scores_cleanliness         

In [264]:
ARBNB_Portland_dropna.to_csv("Resources/cleaned_ARBNB.csv")

# ZILLOW DATA

In [265]:
# Store filepath in a variable
ZLLW = "Resources/portland_housing.csv"

In [266]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ZLLW_df = pd.read_csv(ZLLW, encoding="ISO-8859-1")

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20,50,161,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [267]:
# Show just the header
ZLLW_df.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,brokerageName,dateSold,daysOnZillow,description,favoriteCount,homeStatus,homeType,lastSoldPrice,latitude,livingArea,longitude,lotSize,pageViewCount,postingContact/name,price,priceHistory/0/attributeSource/infoString1,priceHistory/0/attributeSource/infoString2,priceHistory/0/attributeSource/infoString3,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/date,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/pricePerSquareFoot,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/showCountyLink,priceHistory/0/source,priceHistory/0/time,priceHistory/1/attributeSource/infoString1,priceHistory/1/attributeSource/infoString2,priceHistory/1/attributeSource/infoString3,priceHistory/1/date,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/pricePerSquareFoot,priceHistory/1/showCountyLink,priceHistory/1/source,priceHistory/1/time,priceHistory/2/attributeSource/infoString1,priceHistory/2/attributeSource/infoString2,priceHistory/2/attributeSource/infoString3,priceHistory/2/date,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/pricePerSquareFoot,priceHistory/2/showCountyLink,priceHistory/2/source,priceHistory/2/time,priceHistory/3/attributeSource/infoString2,priceHistory/3/date,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/pricePerSquareFoot,priceHistory/3/showCountyLink,priceHistory/3/source,priceHistory/3/time,priceHistory/4/attributeSource/infoString2,priceHistory/4/date,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/pricePerSquareFoot,priceHistory/4/showCountyLink,priceHistory/4/source,priceHistory/4/time,priceHistory/5/attributeSource/infoString2,priceHistory/5/date,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/pricePerSquareFoot,priceHistory/5/showCountyLink,priceHistory/5/source,priceHistory/5/time,priceHistory/6/attributeSource/infoString2,priceHistory/6/date,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/pricePerSquareFoot,priceHistory/6/showCountyLink,priceHistory/6/source,priceHistory/6/time,priceHistory/7/attributeSource/infoString2,priceHistory/7/date,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/pricePerSquareFoot,priceHistory/7/showCountyLink,priceHistory/7/source,priceHistory/7/time,priceHistory/8/attributeSource/infoString2,priceHistory/8/date,priceHistory/8/event,priceHistory/8/postingIsRental,priceHistory/8/price,priceHistory/8/priceChangeRate,priceHistory/8/pricePerSquareFoot,priceHistory/8/showCountyLink,priceHistory/8/source,priceHistory/8/time,priceHistory/9/attributeSource/infoString2,priceHistory/9/date,priceHistory/9/event,priceHistory/9/postingIsRental,priceHistory/9/price,priceHistory/9/priceChangeRate,priceHistory/9/pricePerSquareFoot,priceHistory/9/showCountyLink,priceHistory/9/source,priceHistory/9/time,priceHistory/10/attributeSource/infoString2,priceHistory/10/date,priceHistory/10/event,priceHistory/10/postingIsRental,priceHistory/10/price,priceHistory/10/priceChangeRate,priceHistory/10/showCountyLink,priceHistory/10/source,priceHistory/10/time,propertyTaxRate,rentZestimate,rentalApplicationsAcceptedType,resoFacts/accessibilityFeatures/0,resoFacts/accessibilityFeatures/1,resoFacts/appliances/0,resoFacts/appliances/1,resoFacts/appliances/2,resoFacts/appliances/3,resoFacts/appliances/4,resoFacts/appliances/5,resoFacts/appliances/6,resoFacts/applianc

In [268]:
column_list2 = ZLLW_df.columns.values.tolist()

column_list2

['ï»¿abbreviatedAddress',
 'address/city',
 'address/zipcode',
 'bathrooms',
 'bedrooms',
 'brokerageName',
 'dateSold',
 'daysOnZillow',
 'description',
 'favoriteCount',
 'homeStatus',
 'homeType',
 'lastSoldPrice',
 'latitude',
 'livingArea',
 'longitude',
 'lotSize',
 'pageViewCount',
 'postingContact/name',
 'price',
 'priceHistory/0/attributeSource/infoString1',
 'priceHistory/0/attributeSource/infoString2',
 'priceHistory/0/attributeSource/infoString3',
 'priceHistory/0/buyerAgent/name',
 'priceHistory/0/buyerAgent/photo/url',
 'priceHistory/0/buyerAgent/profileUrl',
 'priceHistory/0/date',
 'priceHistory/0/event',
 'priceHistory/0/postingIsRental',
 'priceHistory/0/price',
 'priceHistory/0/priceChangeRate',
 'priceHistory/0/pricePerSquareFoot',
 'priceHistory/0/sellerAgent/name',
 'priceHistory/0/sellerAgent/photo/url',
 'priceHistory/0/sellerAgent/profileUrl',
 'priceHistory/0/showCountyLink',
 'priceHistory/0/source',
 'priceHistory/0/time',
 'priceHistory/1/attributeSource/i

In [269]:
ZLLW_df.dtypes

ï»¿abbreviatedAddress     object
address/city              object
address/zipcode            int64
bathrooms                float64
bedrooms                 float64
                          ...   
yearBuilt                float64
zestimate                float64
zestimateHighPercent     float64
zestimateLowPercent      float64
zpid                       int64
Length: 348, dtype: object

In [270]:
ZLLW_df2=ZLLW_df[[
    'ï»¿abbreviatedAddress',
     'address/city',
     'address/zipcode',
     'bathrooms',
     'bedrooms',
     #'brokerageName',
     #'dateSold', - not right format - will use priceHistory/0/date
     'daysOnZillow',
     #'description',
     #'favoriteCount',
     'homeStatus',
     'homeType',
     #'lastSoldPrice',  -   decided to just use priceHistory/0/price
     'latitude',
     'livingArea',
     'longitude',
     'lotSize',
     #'pageViewCount', - not enough data here
     #'postingContact/name',
     #'price',-   decided to just use priceHistory/0/price
     #'priceHistory/0/attributeSource/infoString1',
     #'priceHistory/0/attributeSource/infoString2',
     #'priceHistory/0/attributeSource/infoString3',
     #'priceHistory/0/buyerAgent/name',
     #'priceHistory/0/buyerAgent/photo/url',
     #'priceHistory/0/buyerAgent/profileUrl',
     'priceHistory/0/date',
     'priceHistory/0/event',
     #'priceHistory/0/postingIsRental',
     'priceHistory/0/price',
     #'priceHistory/0/priceChangeRate',
     'priceHistory/0/pricePerSquareFoot',
     #'priceHistory/0/sellerAgent/name',
     #'priceHistory/0/sellerAgent/photo/url',
     #'priceHistory/0/sellerAgent/profileUrl',
     #'priceHistory/0/showCountyLink',
     #'priceHistory/0/source',
     #'priceHistory/0/time',
     #'propertyTaxRate',
     'rentZestimate',
     'resoFacts/atAGlanceFacts/0/factValue',
     'resoFacts/basement', 
     'resoFacts/heating/0',
     #'resoFacts/atAGlanceFacts/2/factLabel', just shows heating in every row - no value
     #'resoFacts/atAGlanceFacts/2/factValue',
     'resoFacts/cooling/0',
     #'resoFacts/atAGlanceFacts/3/factLabel', just shows heating in every row - no value
     #'resoFacts/atAGlanceFacts/3/factValue',
     'taxAssessedValue',
     'taxAssessedYear',
     #'taxHistory/0/taxIncreaseRate',
     #'taxHistory/0/taxPaid',
     #'taxHistory/0/time',
     #'taxHistory/0/value',
     #'taxHistory/0/valueIncreaseRate',
     'url',
     'yearBuilt',
     #'zestimate',
     #'zestimateHighPercent',
     #'zestimateLowPercent',
     #'zpid'
]].copy()
pd.set_option('display.max_columns', None)
ZLLW_df2.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,daysOnZillow,homeStatus,homeType,latitude,livingArea,longitude,lotSize,priceHistory/0/date,priceHistory/0/event,priceHistory/0/price,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/cooling/0,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,RECENTLY_SOLD,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,5/21/2021,Sold,315400,175.0,1950.0,Townhouse,Finished,Forced air,NaN,273960.0,2020.0,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,53.0,RECENTLY_SOLD,SINGLE_FAMILY,45.547585,1518.0,-122.453224,3484.0,4/23/2021,Sold,400000,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Central Air,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,Sold,512000,188.0,1995.0,Ranch,Daylight,Forced Air,Wall Unit(s),397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,6/4/2021,Sold,348000,286.0,1695.0,Ranch,Crawl Space,Forced Air,NaN,269520.0,2017.0,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,RECENTLY_SOLD,APARTMENT,45.511238,2400.0,-122.431534,8545.0,6/1/2021,Sold,510000,213.0,1395.0,Apartment,NaN,Other,Central,421980.0,2020.0,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0


In [271]:
ZLLW_df3=ZLLW_df2.rename(columns={"ï»¿abbreviatedAddress": "Address", "address/city": "City","address/zipcode": "Zipcode",'priceHistory/0/price': "price",'priceHistory/0/date': "date_sold"})
ZLLW_df3.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeStatus,homeType,latitude,livingArea,longitude,lotSize,date_sold,priceHistory/0/event,price,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/cooling/0,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,RECENTLY_SOLD,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,5/21/2021,Sold,315400,175.0,1950.0,Townhouse,Finished,Forced air,NaN,273960.0,2020.0,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,53.0,RECENTLY_SOLD,SINGLE_FAMILY,45.547585,1518.0,-122.453224,3484.0,4/23/2021,Sold,400000,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Central Air,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,Sold,512000,188.0,1995.0,Ranch,Daylight,Forced Air,Wall Unit(s),397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,6/4/2021,Sold,348000,286.0,1695.0,Ranch,Crawl Space,Forced Air,NaN,269520.0,2017.0,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,RECENTLY_SOLD,APARTMENT,45.511238,2400.0,-122.431534,8545.0,6/1/2021,Sold,510000,213.0,1395.0,Apartment,NaN,Other,Central,421980.0,2020.0,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0


In [313]:
#filter for only sold homes so we have accurate price
Zillow_Sold=ZLLW_df3.loc[(ZLLW_df3['priceHistory/0/event']=="Sold")]
Zillow_Sold['homeStatus'].value_counts()

RECENTLY_SOLD      25207
FOR_SALE              14
PRE_FORECLOSURE        2
FOR_RENT               2
FORECLOSED             1
Name: homeStatus, dtype: int64

In [314]:
Zillow_Sold2=Zillow_Sold.loc[(Zillow_Sold['homeStatus']=="RECENTLY_SOLD")]
Zillow_Sold2.head()
Zillow_Sold2['homeStatus'].value_counts()

RECENTLY_SOLD    25207
Name: homeStatus, dtype: int64

In [315]:
#drop homeStatus and priceHistory/0/event now that they both only show sold homes
zillow_df=Zillow_Sold2.drop(columns=['homeStatus', 'priceHistory/0/event'])
zillow_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/cooling/0,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,5/21/2021,315400,175.0,1950.0,Townhouse,Finished,Forced air,NaN,273960.0,2020.0,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,53.0,SINGLE_FAMILY,45.547585,1518.0,-122.453224,3484.0,4/23/2021,400000,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Central Air,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,512000,188.0,1995.0,Ranch,Daylight,Forced Air,Wall Unit(s),397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,6/4/2021,348000,286.0,1695.0,Ranch,Crawl Space,Forced Air,NaN,269520.0,2017.0,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,APARTMENT,45.511238,2400.0,-122.431534,8545.0,6/1/2021,510000,213.0,1395.0,Apartment,NaN,Other,Central,421980.0,2020.0,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0


In [316]:
#determine how many home types there are to see if we need an other category - no other category needed less than 10 types
zillow_df['homeType'].value_counts()

SINGLE_FAMILY        21982
CONDO                 1523
TOWNHOUSE             1494
APARTMENT              193
HOME_TYPE_UNKNOWN       15
Name: homeType, dtype: int64

In [317]:
# convert dates from object to date
zillow_df["date_sold"]=zillow_df["date_sold"].apply(pd.to_datetime)
zillow_df.dtypes

Address                                         object
City                                            object
Zipcode                                          int64
bathrooms                                      float64
bedrooms                                       float64
daysOnZillow                                   float64
homeType                                        object
latitude                                       float64
livingArea                                     float64
longitude                                      float64
lotSize                                        float64
date_sold                               datetime64[ns]
price                                            int64
priceHistory/0/pricePerSquareFoot              float64
rentZestimate                                  float64
resoFacts/atAGlanceFacts/0/factValue            object
resoFacts/basement                              object
resoFacts/heating/0                             object
resoFacts/

In [335]:
#drop na and 0 for yearBuilt column
zillow2_df = zillow_df[zillow_df['yearBuilt'].notna()]
zillow3_df=zillow2_df.loc[(zillow2_df['yearBuilt']!=0)]

#change year built from float to datetime
zillow3_df.yearBuilt=pd.to_datetime(zillow3_df["yearBuilt"],format='%Y')

In [342]:
#drop na and 0 for yearBuilt column
zillow4_df = zillow3_df[zillow3_df['taxAssessedYear'].notna()]
zillow5_df=zillow4_df.loc[(zillow4_df['taxAssessedYear']!=0)]

#change tax assessed year from float to datetime
zillow5_df.taxAssessedYear=pd.to_datetime(zillow5_df["taxAssessedYear"],format='%Y')
zillow5_df.dtypes
zillow5_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/cooling/0,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,2021-05-21,315400,175.0,1950.0,Townhouse,Finished,Forced air,NaN,273960.0,2020-01-01,https://www.zillow.com/homedetails/2860-NE-Blo...,2007-01-01
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,2021-06-04,512000,188.0,1995.0,Ranch,Daylight,Forced Air,Wall Unit(s),397560.0,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,Ranch,Crawl Space,Forced Air,NaN,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,APARTMENT,45.511238,2400.0,-122.431534,8545.0,2021-06-01,510000,213.0,1395.0,Apartment,NaN,Other,Central,421980.0,2020-01-01,https://www.zillow.com/homedetails/25-65-NW-20...,1967-01-01
5,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,7000.0,2021-06-01,404200,351.0,1800.0,SingleFamily,NaN,Forced air,NaN,309260.0,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01


In [355]:
zillow6_df=zillow5_df.rename(columns={'resoFacts/cooling/0': "Cooling", 'resoFacts/heating/0': "Heating","resoFacts/atAGlanceFacts/0/factValue": "Type",'resoFacts/basement': "Basement"})
zillow6_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,priceHistory/0/pricePerSquareFoot,rentZestimate,Type,Basement,Heating,Cooling,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,2021-05-21,315400,175.0,1950.0,Townhouse,Finished,Forced air,NaN,273960.0,2020-01-01,https://www.zillow.com/homedetails/2860-NE-Blo...,2007-01-01
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,2021-06-04,512000,188.0,1995.0,Ranch,Daylight,Forced Air,Wall Unit(s),397560.0,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,Ranch,Crawl Space,Forced Air,NaN,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,APARTMENT,45.511238,2400.0,-122.431534,8545.0,2021-06-01,510000,213.0,1395.0,Apartment,NaN,Other,Central,421980.0,2020-01-01,https://www.zillow.com/homedetails/25-65-NW-20...,1967-01-01
5,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,7000.0,2021-06-01,404200,351.0,1800.0,SingleFamily,NaN,Forced air,NaN,309260.0,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01


In [361]:
#figure out how to clean column
zillow6_df['Cooling'].value_counts()

#need to add central air and central

#need to add wall unit with wall

#drop everything below 150

Central Air                        11330
None                                1770
Heat Pump                           1226
Central                              946
Air Conditioning Ready               667
Other                                279
Wall Unit(s)                         267
Window Unit(s)                       251
ENERGY STAR Qualified Equipment      111
Wall                                  38
Exhaust Fan                           18
Heat Exchanger                        12
Refrigerator                          11
Solar                                  1
Geothermal                             1
Evaporative                            1
Name: Cooling, dtype: int64

In [357]:
# Determine which values to replace if counts are less than ...?
replace_application=list(application_type_counts[application_type_counts < 150].index)

# Replace in dataframe
for i in replace_application:
    zillow6.Cooling = zillow6.Cooling.replace(i,"Other")
    
# Check to make sure binning was successful
zillow6.Cooling.value_counts()

NameError: name 'application_type_counts' is not defined

In [362]:
#figure out how to clean column
zillow6_df['Heating'].value_counts()

Forced Air                         13889
Forced air                          2543
Forced Air - 95+%                   1776
Forced Air - 90%                    1304
Baseboard                            883
Heat Pump                            608
Zoned                                482
Other                                422
ENERGY STAR Qualified Equipment      189
Heat pump                            187
Mini Split                           164
Radiant                              162
Hot Water                             92
Wall Furnace                          92
Ceiling                               71
Wall                                  56
Floor Furnace                         55
Wood Stove                            42
Active Solar                          33
Ductless                              29
None                                  18
Gas                                    9
Electric                               9
Gravity                                7
Passive Solar   

In [363]:
#figure out how to clean column
zillow6_df['Type'].value_counts()

Single Family Residence                    16395
SingleFamily                                3284
Condominium                                 1008
Ranch                                        785
Townhouse                                    549
Attached, Townhouse                          472
Attached                                     210
Condo                                        199
Apartment                                    189
Condominium, Townhouse                       159
Farm                                          60
Condominium, Detached Condo                   12
Unknown                                        7
Single Family Residence, Townhouse             6
Mixed Use                                      5
Single Family Residence, Farm                  4
Single Family Residence, Detached Condo        4
Residence                                      2
Cooperative                                    1
MobileManufactured                             1
Single Family Reside

In [364]:
#figure out how to clean column
zillow6_df['Basement'].value_counts()

Crawl Space                                  9892
Finished                                     1680
None                                         1146
Finished,Full                                 722
Unfinished                                    628
                                             ... 
Finished,Exterior Entry,Other                   1
Finished, Apartment                             1
Crawl Space,Unfinished,Partially Finished       1
Crawl Space, Dirt Floor                         1
Crawl Space,Full,Storage Space                  1
Name: Basement, Length: 203, dtype: int64

In [365]:
zillow6_df.isna().sum()

Address                                 0
City                                    0
Zipcode                                 0
bathrooms                              41
bedrooms                              317
daysOnZillow                            0
homeType                                0
latitude                                0
livingArea                             26
longitude                               0
lotSize                              1927
date_sold                               0
price                                   0
priceHistory/0/pricePerSquareFoot      27
rentZestimate                         193
Type                                    0
Basement                             3765
Heating                               218
Cooling                              6425
taxAssessedValue                        2
taxAssessedYear                         0
url                                     1
yearBuilt                               0
dtype: int64

In [276]:
ZLLW_df3.dropna()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeStatus,homeType,latitude,livingArea,longitude,lotSize,date_sold,priceHistory/0/event,price,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/cooling/0,taxAssessedValue,taxAssessedYear,url,yearBuilt
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,Sold,512000,188.0,1995.0,Ranch,Daylight,Forced Air,Wall Unit(s),397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
6,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,25.0,RECENTLY_SOLD,SINGLE_FAMILY,45.498184,2036.0,-122.451332,6969.0,5/21/2021,Sold,478200,235.0,2250.0,SingleFamily,Finished,Heat pump,Central,373030.0,2020.0,https://www.zillow.com/homedetails/110-NW-Will...,1978.0
9,3569 SW Mckinley St,Gresham,97080,3.0,3.0,41.0,RECENTLY_SOLD,SINGLE_FAMILY,45.475353,2209.0,-122.468307,5227.0,5/5/2021,Sold,550000,249.0,2300.0,Single Family Residence,Crawl Space,Forced Air,Central Air,453610.0,2020.0,https://www.zillow.com/homedetails/3569-SW-Mck...,2017.0
13,705 NE 18th Ct,Gresham,97030,1.0,3.0,75.0,RECENTLY_SOLD,SINGLE_FAMILY,45.510002,1288.0,-122.423157,9147.0,4/1/2021,Sold,330000,256.0,1895.0,Ranch,Crawl Space,Forced Air,Central Air,323820.0,2020.0,https://www.zillow.com/homedetails/705-NE-18th...,1969.0
15,2349 SW 19th Ct,Gresham,97080,3.0,3.0,53.0,RECENTLY_SOLD,SINGLE_FAMILY,45.482521,1772.0,-122.457100,11324.0,4/23/2021,Sold,465000,262.0,2200.0,SingleFamily,Finished,Forced air,Central,399990.0,2020.0,https://www.zillow.com/homedetails/2349-SW-19t...,1974.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25719,8675 SW Stono Dr,Tualatin,97062,3.0,4.0,340.0,RECENTLY_SOLD,SINGLE_FAMILY,45.354996,2500.0,-122.766136,8712.0,8/13/2020,Sold,468000,187.0,2894.0,Single Family Residence,Crawl Space,Forced Air,Central Air,439970.0,2019.0,https://www.zillow.com/homedetails/8675-SW-Sto...,1992.0
25722,7440 SW Hart Pl,Beaverton,97008,3.0,4.0,353.0,RECENTLY_SOLD,SINGLE_FAMILY,45.466007,2703.0,-122.812218,13068.0,7/31/2020,Sold,499000,185.0,2510.0,Single Family Residence,"Daylight,Finished",Forced Air,Central Air,463560.0,2019.0,https://www.zillow.com/homedetails/7440-SW-Har...,1974.0
25723,15455 SW Gleneden Dr,Beaverton,97007,3.0,4.0,357.0,RECENTLY_SOLD,SINGLE_FAMILY,45.463707,2145.0,-122.835503,6534.0,7/27/2020,Sold,525000,245.0,2795.0,Single Family Residence,Crawl Space,Forced Air,Central Air,484840.0,2019.0,https://www.zillow.com/homedetails/15455-SW-Gl...,1989.0
25725,7050 SW 152nd Ave,Beaverton,97007,2.0,3.0,360.0,RECENTLY_SOLD,SINGLE_FAMILY,45.468781,1924.0,-122.833855,14374.0,7/24/2020,Sold,485000,252.0,2562.0,Single Family Residence,Crawl Space,Forced Air,Central Air,410120.0,2019.0,https://www.zillow.com/homedetails/7050-SW-152...,1981.0


In [277]:
ZLLW_df3.dtypes

Address                                  object
City                                     object
Zipcode                                   int64
bathrooms                               float64
bedrooms                                float64
daysOnZillow                            float64
homeStatus                               object
homeType                                 object
latitude                                float64
livingArea                              float64
longitude                               float64
lotSize                                 float64
date_sold                                object
priceHistory/0/event                     object
price                                     int64
priceHistory/0/pricePerSquareFoot       float64
rentZestimate                           float64
resoFacts/atAGlanceFacts/0/factValue     object
resoFacts/basement                       object
resoFacts/heating/0                      object
resoFacts/cooling/0                     

In [278]:
ZLLW_df3.to_csv("Resources/cleaned_zillow.csv")